**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024

# Uranium Mill Network Dynamics Simulation Report
$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Alyssa Puglisi <br>
**Course:** Engy-4390 Nuclear Systems Design and Analysis  <br>
**Mentor:** Prof. Valmor F. de Almeida

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**18Nov2024**

[Project Repository on `GitHub`](https://github.com/dpploy/engy-4390/tree/main/projects/2024/u-mill)

---
## Table of Contents<a id="toc"></a>
* [Problem Statment](#prob)
* [Introduction](#intro)
   * [Simple Solvex](#simple_solvex)
   * [White Mesa's Solvex](#wm_solvex)
* [Source Code](#src)
* [Objectives](#obj)
* [Tools](#tools)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Solvent Extraction Module](#IPM)
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

In [20]:
from cortix import ReactionMechanism

## [Problem Statement](#toc)<a id="prob"></a>

Design a dynamic system that accurately models the mass transfer with multiple phases occuring in the solvent extraction then stripping process during White Mesa's uranium milling. 

In [21]:
rxn_mech = ReactionMechanism(file_name='solvex-stripping-rxn-mechanism.txt', order_species=True)
rxn_mech.md_print()

#help(rxn_mech)
#TODO C11H20O2 is only showing up as C1 H2 O2 ??

8 **Species:** 
 ${\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}} \quad  {\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)} \quad  {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}} \quad  {\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}$

4 **Reactions:** 
 \begin{align*} 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ 2.0\,{[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ 2.0\,{[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ 2.0\,{\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
\end{align*} 


The amine, on contact with a sulphuric acid solution, is converted into the sulphate or bisulphate salt.

The actual extraction reactions are complex. However, in a simplified way, the extraction can be depicted as an anion exchange where the anionic uranium complex from the aqueous solution replaces the sulphate or bisulphate ion in the organic phase. 

The data check below shows that the reactions within solvent extraction are coupled. We can see that the set of reaction mechanism are not full rank, indicating that the concentration of individual species are dependent on multiple reactions and multiple species.

In [26]:
'''Data Check'''
print('Is mass conserved?', rxn_mech.is_mass_conserved())
rxn_mech.rank_analysis(verbose=True, tol=1e-8)
print('S=\n', rxn_mech.stoic_mtrx)

Is mass conserved? False
# reactions =  4
# species   =  8
rank of S =  4
S is full rank.
S=
 [[-1.  0.  0.  0. -2.  0.  1.  0.]
 [-1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  0.  1. -1.  0.  0. -1.  1.]
 [ 0.  2.  0. -1.  0.  0. -2.  1.]]


Each species concentration must then be solved for co-currently with eachother. This is key to developing the dynamic model of White Mesa's solvent extraction.

## [Introduction](#toc)<a id="intro"></a>


### [Simple Solvex](#toa)<a id="simple_solvex"></a>

Solvent extraction, also known as liquid-liquid extraction, is the partition of metal complexes from two immiscible liquids. The two most common liquids used are on aqueous and one organic. Solvent extraction works on the exploitation of the relative solubility of the solute (metal complexes) in the two phases. The chemical potentials of each input make up the driving force of the transfer. Increased stabilitiy of the products as compared to the initial solution is necessary to increase the driving force. The solute enters the extraction system in the aqeuous feed stream and is extracted by the incoming solvent/organic stream. The feed is reduced and exits the extraction system as the raffinate stream; the solvent stream absorbs and exits the extraction system as the extract stream. Solvent extraction products do need further purification by methods such as percipitation, distillation, etc.

Solvent extraction is chosen as the necessary separation technique due to the high precision and accuracy of the separation. All based on the proper selection of the solvent, individual and specific complexes can be removed from the feed stream with substancial levels of exactness. Additional extraction tanks can be added to the system to see extraction percentages close to 100%. Morover, solvent extraction plus a distillation or percipitation process is chosen over just one of the former because of the increased simplicity of the distillation/precipitation process after solvent partitioning. All the benefits of solvent extraction are based on the choice of solvent.

The optimal choice solvent needs these three critical charactisterics:

* high affinitiy for the solute with low/no affinity for the carrier
* different in density when compared to the feed/carrier
* solute is easily recoverable from solvent

In an ideal world, other beneficial qualities of the choice solvent are:

* inexpensive
* non-toxic to humans and the environment
* non-volatile (low vapor pressure)
* non-flammable
* non-reactive
* non-corrosive

### [White Mesa's Solvex](#toa)<a id="wm_solvex"></a>

Reaction of Solvent Extraction:
<br></br>
<center>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub><sup>4-</sup> + 2(R<sub>3</sub>NH<sup>+</sup>)<sub>2</sub>SO<sub>4</sub> → (R<sub>3</sub>NH<sup>+</sup>)<sub>4</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub> + 2SO<sub>4</sub><sup>2-</sup></center>

Reaction of Stripping:
<br></br>
<center>(R<sub>3</sub>NH<sup>+</sup>)<sub>4</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub> + 4NH<sub>4</sub>OH ⇌ 4R<sub>3</sub>N + 4H<sub>2</sub>O + (NH<sub>4</sub>)<sub>2</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>2</sub> + (NH<sub>4</sub>)<sub>2</sub>SO</center>

<img src="pics/mixer-settler.jpg" alt="Figure 1: Mixer Settler Machine" width="350"/>

The uranium solvent extraction circuit consists of:

(a) Four extraction stages, 

(b) One organic washing stage,

(c) Four stripping stages, 

(d) One organic regeneration stage.

In White Mesa, the organic phase consists of Alamine 336 diluted with kerosene modified with isodecanol. Alamine 336 is made almost entirely of Trioctylamine or TOA for short.
<br></br>
<center>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub><sup>4-</sup> + 2(R<sub>3</sub>NH<sup>+</sup>)<sub>2</sub>SO<sub>4</sub> → (R<sub>3</sub>NH<sup>+</sup>)<sub>4</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub> + 2SO<sub>4</sub><sup>2-</sup></center>

From this equation the UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub><sup>4-</sup> is the feed stream from the filtration module. The product of solvent extraction is (R<sub>3</sub>NH<sup>+</sup>)<sub>4</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub> with the other product exiting as a waste stream. One problem with this technique is the extraction of molybdenum is preferred to uranium and must be removed. This solution can be introduced to a stripping column to increase the purity of the solution through the removal of molybdenum. The selective stripping of uranium from the solution can be seen with the chemical reaction below.
<br></br>
<center>(R<sub>3</sub>NH<sup>+</sup>)<sub>4</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub> + 4NH<sub>4</sub>OH ⇌ 4R<sub>3</sub>N + 4H<sub>2</sub>O + (NH<sub>4</sub>)<sub>2</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>2</sub> + (NH<sub>4</sub>)<sub>2</sub>SO</center>

Solvent extraction occurs in mixer-settler vessels where the organic and aqueous phases are agitated in a mixer then deposited into a settling tank to phase split. The aqueous phase will sink to the bottom with the organic phase rising to the top. The organic phase is then sent to the previous stage with the aqueous phase being passed to the next stage. This results in the first mixer-settler vessel delivering the product to the stripping circuit with the final mixer-settler vessel delivering a waste stream to the tailings area.

After solvent extraction the stripping circuit begins with an acidic water wash. Immediately afterwards the organic phase is deposited into another bank of mixer-settler vessels where an acidic sodium chloride solution is used to extract uranyl ions. The stripping circuit will remove nearly all of the uranium from the organic phase resulting in a product of (NH<sub>4</sub>)<sub>2</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>2</sub> which is ready to be fed into the precipitation stage where the sulfate will be removed from the compound to be precipitated into U<sub>3</sub>O<sub>8</sub>.

## [Source](#toc)<a id='src'></a>

Documentation options:

* Live in this notebook run on code cell: help(src.stage)
* On the web: [Solvent Extraction Source](https://github.com/dpploy/engy-4390/blob/main/projects/2024/u-mill/src/solvex.py)

In [23]:
'''For help purposes'''
import src.solvex

## [Objectives](#toc)<a id="obj"></a>

- Python code will be used to develop dynamic comprehensive systems that accurately represent the solvent extraction process. This is a non-equilibrium, reactive mass transfer scenario. The analysis involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis.
- Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  
- A flowsheet based on White Mesa will be developed as the basis for our modules



## [Tools](#toc)<a id="tools"></a>

For this project we will be utilizing GitHub to store a remote repository of our python code and project report. GitHub is a version control software that programmers use to share and work on coding projects together. This allows for users to view the contribution history easily and rollback changes if needed. GitHub interfaces with git to create a graphical user interface for the git repository. At its core, a GitHub repository is a mirror of a git repository with any changes being submitted through GitHub automatically being sent to the git repository as well.

<img src="github-logo.jpg" alt="Figure 2: Github Logo" width="150"/>

Cortix is a library for Python that was created by Dr. Almeida and his research group to simplify the process of producing dynamic system models. The full open-source code can be found on Github, where anyone can download and use the library and its available unit operation modules. The Github repository also contains examples that use the code to simulate various chemical interactions while delivering time dependent data. 

<img src="cortix-logo.jpg" alt="Figure 3: Cortix Logo" width="150"/>

Jupyter Notebook is also open-source, and it is a way to integrate ordinary text with multiple programming languages such as Python. Python is a widely used programming language that has engineering modeling applications. This is a popular language for dynamic modelling and has been in use for over thirty years. It is commonplace in many programs used in the nuclear industry, making it a great candidate for this project. 

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

#### Assumptions 
+ ideal flow mixing 
+ ideal solution


-  First, a series of mass balances and component balances are made based on the desired input of 1 ton of raw material.
- These balances then take into account the chemical kinetics and related rates required in the operation of White Mesa's chemical reactions and fluid flow operations
- Once baseline models of each individual process exist, they are then programmed into 5 separate but connected modules within Cortix
- Each process will be capable of interacting with the modules connected to it on either end, allowing them to share data such as the incoming and outgoing streams from one process to the next

|     **Module**        |      **Inputs**      |      **Outputs**       |     **Legend**     |
|:----------------------|:---------------------|:-----------------------|:-------------------|
| Solvent Extraction             |m<sub>aef</sub>, m<sub>oef</sub>, m<sub>voii</sub>, m<sub>oii</sub>, m<sub>vaii</sub>, m<sub>aii</sub>        | m<sub>ap</sub>, m<sub>op</sub>, m<sub>voio</sub>, m<sub>oio</sub>, m<sub>vaio</sub>, m<sub>aio</sub>            |m<sub>aef</sub> = mass flow rate of aqueous external feed, m<sub>oef</sub> = mass flow rate of organic external feed, m<sub>voii</sub> = mass flow rate of vapor organic inter-stage inflow, m<sub>oii</sub> = mass flow rate of organic inter-stage inflow, m<sub>vaii</sub> = mass flow rate of vapor aqueous inter-stage inflow, m<sub>aii</sub> = mass flow rate of aqueous inter-stage inflow, m<sub>ap</sub> = mass flow rate of aqeuous product,m<sub>op</sub> = mass flow rate of organic product, m<sub>voio</sub> = mass flow rate of vapor organic inter-stage outflow, m<sub>oio</sub> = mass flow rate of organic inter-stage outflow, m<sub>vaio</sub> = mass flow rate of vapor aqueous inter-stage outflow, m<sub>aio</sub> = mass flow rate of aqueous inter-stage outflow           |             |        |           |        |

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

<img src="insitu-diagram.png" alt="Figure 4: Insitu Uranium Mining" width="380"/>

hen this project was initially proposed, we were expecting to model an In-Situ style uranium mill. This is due to the advantage of causing less ground disturbance for deeper reservoirs than with a traditional open pit mine, as well as the ability to be a one site process due to the removal of the ore crushing and transportation steps of the process. Even though this process shows promising signs, there is not much publicly available information on the operations of an In-Situ mine. This is most likely due to it being a newer method of uranium recovery with proprietary knowledge not yet being shared. As a result, this presented a challenge of finding a better option that still has realistic and useful applications. This was what led us to the option of modelling White Mesa’s heap leaching method. 

## [Initialization of Solvent Extraction](#toc)<a id="IPM"></a>

```python   
                                      |
                                      |  Extraction Feed (from Decantation-Filtration filtrate)
                                      |
                                      v
                              |----------------|
                              |                |
         Organic Feed ------->|                |------> Organic Product (to Scrubbing internal)
         (internal source)    |    Solvent     |
                              |   Extraction   |
         Raffinate <----------|                |<------ Scrub Raffinate (from Scrubbing internal)
         Stream (to CCD Bank) |                |
                              |----------------|
          Organic Product <---|                |<------ Organic Feed (Organic Product from Solv. Extr. internal)
          (to Strip internal) |                |
                              |   Scrubbing    |
           Scrub Feed ------->|                |-------> Scrub Raffinate (to Solvent Extraction internal)
           (internal source)  |                |
                              |----------------|
          Organic Feed ------>|                |-------> Organic Regeneration (to Solvent Extraction not done)
        (from Scrub internal) |   Stripping    |
                              |                |<------ Stripping Feed (internal source)
                              |________________|<------ Stripping Feed (from Precipitation not implemented)
                                      |
                                      |
                                      |
                                      v
                               Stripping Product (Precipitation feed)
```

## [References](#toc)<a id="refs"></a>

- [1] “35 No. 9 Uranium Extraction Technology - IAEA.” Uranium Extraction Technology, IAEA, 1993, https://www-pub.iaea.org/MTCD/Publications/PDF/trs359_web.pdf. 
- [2] "31 No. 3 Manual on Laboratory Testing for Uranium Ore Precessing - IAEA." Manual on Laboratory Testing for Uranium Ore Precessing, IAEA, 1990, https://www-pub.iaea.org/MTCD/Publications/PDF/trs313_web.pdf

#### to do next
+ add the results/plots
+ show usecase and stand alone solvex
+ explain repo layout right after ReadMe
+ explain the system before the problem statement
+ should start w/ ReadME file that is the most broad explanation
+ start presentation w/ VERY broad problem statement